In [2]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR_l2": LogisticRegression(penalty="l2",random_state=1234),
    "GNB": GaussianNB(),
    "LR": LogisticRegression(penalty="none", random_state=1234) 
}

In [3]:
datasets = ['breast', 'campus', 'churn', 'climate',
            'compas', 'diabetes', 'german', 'heart',
            'stroke', 'student', 'water', 'bankruptcy']

In [4]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
n_datasets = len(datasets)
n_splits = 10
# repeats 5, splits 2
skf = StratifiedKFold(n_splits=n_splits, random_state=1234)

auc_scores = np.zeros((len(clfs)+1, n_datasets, n_splits))
loss = np.zeros((len(clfs)+1, n_datasets, n_splits))

In [5]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(skf.split(X, y)):
        for clf_id, clf_name in enumerate(clfs):
            clf = clone(clfs[clf_name])

            # if(clf_name == "GNB" and dataset=="compas"):
            #     clf.fit(X.iloc[train], y.iloc[train])
            #     y_preds = clf.predict(X.iloc[test])
            #     fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
            #     auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            # else:
            clf_pipeline = make_pipeline(
                preprocess,
                clf
            )
                
            clf_pipeline.fit(X.iloc[train], y.iloc[train])
            y_preds = clf_pipeline.predict(X.iloc[test])
            fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
            auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            loss[clf_id, data_id, fold_id] = log_loss(y.iloc[test], y_preds)

            

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed w

In [6]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(skf.split(X, y)):

        clf_pipeline = make_pipeline(
                preprocess,
                DecisionListClassifier(random_state=1234)
            )
                
        clf_pipeline.fit(X.iloc[train], y.iloc[train])
        y_preds = clf_pipeline.predict(X.iloc[test])
        fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
        auc_scores[4, data_id, fold_id] = metrics.auc(fpr, tpr)
        loss[4, data_id, fold_id] = log_loss(y.iloc[test], y_preds)
        

In [7]:
auc_scores

array([[[1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ]],

       [[1.        , 1.        , 0.97619048, 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 0.97727273]],

       [[1.        , 1.        , 0.97619048, 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 0.97727273]],

       [[1.        , 1.        , 0.97619048, 0.93181818, 1.        ,
         0.97619048, 0.97619048, 1.        , 0.97727273, 1.        ]],

       [[1.        , 0.94827586, 1.        , 0.93181818, 1.        ,
         1.        , 0.98275862, 1.        , 0.96428571, 1.        ]]])

In [8]:
auc_scores[0,:, 0]

array([1.])

In [9]:
auc_scores[1,:, 0]

array([1.])

In [10]:
auc_scores[2,:, 0]

array([1.])

In [11]:
auc_scores[3,:, 0]

array([1.])

In [12]:
auc_scores[4,:, 0]

array([1.])

In [13]:
np.save('./test_results/auc/auc_results', auc_scores)
np.save('./test_results/auc/auc_losses', loss)